In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re  # Import regular expressions

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the provided URL
driver.get("https://www.goldenscent.com/c/fragrances.html?action=cat&id=3")

# Wait for the page to load
time.sleep(5)  # Adjust the wait time if needed

# Initialize an empty list to store unique hrefs
unique_hrefs = []

# Loop to collect unique hrefs for up to 200 perfumes
while len(unique_hrefs) < 1000:
    # Locate the perfume links and extract their href attributes
    golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    golden_hrefs = [link.get_attribute('href') for link in golden_links]

    # Collect unique hrefs
    for href in golden_hrefs:
        if href not in unique_hrefs:
            unique_hrefs.append(href)
        if len(unique_hrefs) >= 1000:  # Stop once we have 200 unique hrefs
            break

    # Scroll down to load more products if needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Adjust the wait time if needed

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)  
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)  
            product_info[label] = value  

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


                                                 الاسم  السعر النهائي  \
0                             إيف سان لوران ليبر إنتنس            494   
1    جورجيو أرماني إمبريو أرماني سترونجر ويذ يو إنت...            536   
2                               روبرتو كافالي باراديسو            222   
3         فيري إمبورتانت نوتس مجموعة ديو الأفضل مبيعاَ            249   
4            فرزاتشي كريستال نوار أو دو برفيوم - 90 مل            518   
..                                                 ...            ...   
995                                مونتال بينك إكستاسي            573   
996                                  مونتال إنتنس بيبر            435   
997                                   مونتال داي دريمز            426   
998             رويال آزور للعطور ميجامار أو دو برفيوم            150   
999            رويال آزور للعطور بارادايس أو دو برفيوم            150   

                 الماركة        الجنس نوع المنتج شخصية عطرك العائلة العطرية  \
0           اف سان لوران        نسائي       

In [7]:
golden_df

,الاسم,السعر النهائي,الماركة,الجنس,نوع المنتج,شخصية عطرك,العائلة العطرية,الحجم,المكونات,نسبة التركيز,مقدمة العطر,قلب العطر,قاعدة العطر,اللون,سنة الإصدار,Category,نوع المكياج,مفتاح البحث
0,إيف سان لوران ليبر إنتنس,494,اف سان لوران,نسائي,عطور,حيوية,فاكهية,50 مل,برغموت، برتقال، لافندر، ياسمين سامباك، زهرة ال...,أو دو برفيوم,برغموت، برتقال، لافندر,لافندر، ياسمين سامباك، زهرة الأوركيد، زهر البر...,فانيلا، حبوب التونكا، نجيل الهند، عنبر,NaN,NaN,NaN,NaN,NaN
1,جورجيو أرماني إمبريو أرماني سترونجر ويذ يو إنت...,536,جورجيو ارماني,رجالي,عطور,جذابة,أروماتيك,100 مل,التوت الناضج ، ألورد ، المر الشيح ، الباتشولي,أو دو برفيوم,فلفل وردي، العرعر، ورق البنفسج,خزامى، ميرامية، توفي، قرفة,حبوب تونكا، جلد الغزال، كهرمان، فانيليا,ذهبي,2019,NaN,NaN,NaN
2,روبرتو كافالي باراديسو,222,روبرتو كافالي,نسائي,عطور,جذابة,خشبية,50 مل,الحمضيات، اليوسفي، البارغموت، الياسمين، أشجار ...,أو دو برفيوم,حمضيات، اليوسفي، البرغموت,ياسمين,شجرة السرو، صنوبر، إكليل الغار,NaN,2015,عطور,NaN,NaN
3,فيري إمبورتانت نوتس مجموعة ديو الأفضل مبيعاَ,249,فيري إمبورتانت نوتس,رجالي/نسائي,مجموعات,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,فرزاتشي كريستال نوار أو دو برفيوم - 90 مل,518,فرزاتشي,نسائي,عطور,عصرية,زهرية شرقية,90مل,زنجبيل، الحبهان، الفلفل، براعم البرتقال، الفاو...,أو دو برفيوم,زنجبيل، الحبهان، الفلفل،,براعم البرتقال، الفاوانيا، الجاردينيا، جوز الهند,خشب الصندل، المسك، العنبر.,NaN,2004,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,مونتال بينك إكستاسي,573,مونتال,نسائي,عطور,رومانسية,زهرية,100 مل,البرتقال، اليوسفي، الفلفل الأسود، الياسمين، ال...,أو دو برفيوم,البرتقال، اليوسفي، الفلفل الأسود,الياسمين، الورد التركي، السوسن، زهر البرتقال,المسك الأبيض، الفانيليا، التوت، الخوخ,وردي,2014,NaN,NaN,NaN
996,مونتال إنتنس بيبر,435,مونتال,رجالي/نسائي,عطور,جذابة,شرقية,100 مل,الليمون الصقلي، الفلفل الوردي، الفلفل الأسود،م...,أو دو برفيوم,الفلفل الوردي، الفلفل الأسود,مكونات زهرية,العود، الأرز، العنبر,"أسود,ذهبي",2014,NaN,NaN,NaN
997,مونتال داي دريمز,426,مونتال,رجالي/نسائي,عطور,حيوية,زهرية,100 مل,برتقال الماندرين، زهر البرتقال، زهرة تياري، ال...,أو دو برفيوم,برتقال الماندرين، زهر البرتقال، الياسمين,زهر البرتقال، زهرة تياري، خشب الصندل,الفانيليا، جوز الهند,"أزرق,ذهبي",2017,NaN,NaN,NaN
998,رويال آزور للعطور ميجامار أو دو برفيوم,150,رويال آزور للعطور,رجالي/نسائي,عطور,ديناميكية,حمضية,80 مل,NaN,أو دو برفيوم,البرجموت، الليمون,نوتات أروماتيك، الأعشاب البحرية، كالون,المسك، أمبروكسان، الأرز,NaN,NaN,NaN,NaN,NaN


In [9]:
golden_df[golden_df["نوع المنتج"] == "عطور"]

,الاسم,السعر النهائي,الماركة,الجنس,نوع المنتج,شخصية عطرك,العائلة العطرية,الحجم,المكونات,نسبة التركيز,مقدمة العطر,قلب العطر,قاعدة العطر,اللون,سنة الإصدار,Category,نوع المكياج,مفتاح البحث
0,إيف سان لوران ليبر إنتنس,494,اف سان لوران,نسائي,عطور,حيوية,فاكهية,50 مل,برغموت، برتقال، لافندر، ياسمين سامباك، زهرة ال...,أو دو برفيوم,برغموت، برتقال، لافندر,لافندر، ياسمين سامباك، زهرة الأوركيد، زهر البر...,فانيلا، حبوب التونكا، نجيل الهند، عنبر,NaN,NaN,NaN,NaN,NaN
1,جورجيو أرماني إمبريو أرماني سترونجر ويذ يو إنت...,536,جورجيو ارماني,رجالي,عطور,جذابة,أروماتيك,100 مل,التوت الناضج ، ألورد ، المر الشيح ، الباتشولي,أو دو برفيوم,فلفل وردي، العرعر، ورق البنفسج,خزامى، ميرامية، توفي، قرفة,حبوب تونكا، جلد الغزال، كهرمان، فانيليا,ذهبي,2019,NaN,NaN,NaN
2,روبرتو كافالي باراديسو,222,روبرتو كافالي,نسائي,عطور,جذابة,خشبية,50 مل,الحمضيات، اليوسفي، البارغموت، الياسمين، أشجار ...,أو دو برفيوم,حمضيات، اليوسفي، البرغموت,ياسمين,شجرة السرو، صنوبر، إكليل الغار,NaN,2015,عطور,NaN,NaN
4,فرزاتشي كريستال نوار أو دو برفيوم - 90 مل,518,فرزاتشي,نسائي,عطور,عصرية,زهرية شرقية,90مل,زنجبيل، الحبهان، الفلفل، براعم البرتقال، الفاو...,أو دو برفيوم,زنجبيل، الحبهان، الفلفل،,براعم البرتقال، الفاوانيا، الجاردينيا، جوز الهند,خشب الصندل، المسك، العنبر.,NaN,2004,NaN,NaN,NaN
5,مانسيرا روز فانيلا - 120 مل,495,مانسيرا,رجالي/نسائي,عطور,أنثوية,زهرية,120 مل,ليمون كالاباري، حلوى تركية، روز، خشب الأرز، فا...,أو دو برفيوم,ليمون كالاباري,حلوى تركية، روز,خشب الأرز، فانيليا مكثفة، مسك أبيض,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,مونتال بينك إكستاسي,573,مونتال,نسائي,عطور,رومانسية,زهرية,100 مل,البرتقال، اليوسفي، الفلفل الأسود، الياسمين، ال...,أو دو برفيوم,البرتقال، اليوسفي، الفلفل الأسود,الياسمين، الورد التركي، السوسن، زهر البرتقال,المسك الأبيض، الفانيليا، التوت، الخوخ,وردي,2014,NaN,NaN,NaN
996,مونتال إنتنس بيبر,435,مونتال,رجالي/نسائي,عطور,جذابة,شرقية,100 مل,الليمون الصقلي، الفلفل الوردي، الفلفل الأسود،م...,أو دو برفيوم,الفلفل الوردي، الفلفل الأسود,مكونات زهرية,العود، الأرز، العنبر,"أسود,ذهبي",2014,NaN,NaN,NaN
997,مونتال داي دريمز,426,مونتال,رجالي/نسائي,عطور,حيوية,زهرية,100 مل,برتقال الماندرين، زهر البرتقال، زهرة تياري، ال...,أو دو برفيوم,برتقال الماندرين، زهر البرتقال، الياسمين,زهر البرتقال، زهرة تياري، خشب الصندل,الفانيليا، جوز الهند,"أزرق,ذهبي",2017,NaN,NaN,NaN
998,رويال آزور للعطور ميجامار أو دو برفيوم,150,رويال آزور للعطور,رجالي/نسائي,عطور,ديناميكية,حمضية,80 مل,NaN,أو دو برفيوم,البرجموت، الليمون,نوتات أروماتيك، الأعشاب البحرية، كالون,المسك، أمبروكسان، الأرز,NaN,NaN,NaN,NaN,NaN
